This notebook replicates the brain-related analyses. Note: the fMRI data can be downloaded into the data folder from here: https://dataspace.princeton.edu/jspui/handle/88435/dsp01nz8062179

## Import libraries

In [ ]:
import glob
import numpy as np
from nilearn.image import load_img
from nilearn.image import new_img_like
from nilearn.image import index_img
from nilearn.image import concat_imgs
from nilearn.image import mean_img
from brainiak.searchlight.searchlight import Searchlight
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
from scipy.stats import ttest_1samp as ttest

from nilearn import datasets
from nilearn.image import load_img
from nilearn.image import index_img
from nilearn import surface
from nilearn import plotting
%matplotlib inline

def _r2z(r):
    """
    Function that calculates the Fisher z-transformation

    Parameters
    ----------
    r : int or ndarray
        Correlation value

    Returns
    ----------
    result : int or ndarray
        Fishers z transformed correlation value

    """
    with np.errstate(invalid='ignore', divide='ignore'):
        return 0.5 * (np.log(1 + r) - np.log(1 - r))

## Set path

In [ ]:
datadir = '../../../data/processed/'

## Collect permutations

In [ ]:
# ref_img = load_img(datadir+'searchlight_movie/ref.nii.gz')

# for perm in range(100):
#     print('perm #%s' % str(perm))
#     imgs_recall = []
#     for sub in range(1, 18):
#         fpath = glob.glob(datadir+'searchlight_movie/s%s_perm%s_*' % (str(sub), str(perm)))
#         sub_data = np.load(fpath[0])
#         img = new_img_like(ref_img, sub_data.astype(np.float))
#         imgs_recall.append(img)
#     imgs_recall = concat_imgs(imgs_recall)
#     x, y, z, t = imgs_recall.shape
#     statmap = ttest(np.moveaxis(imgs_recall.get_data(), -1, 0), 0).statistic
#     img = new_img_like(ref_img, statmap.astype(np.float))
#     img.to_filename(datadir + 'searchlight_movie/perm%s' % str(perm))

## Load in permutations and sort them

In [ ]:
perms = [load_img(datadir+'searchlight_movie/perm%s.nii' % str(perm)) for perm in range(100)]
perms = concat_imgs(perms)
sorted_perms = np.sort(perms.get_data(), axis=3)

## Load in real data

In [ ]:
ref_img = load_img(datadir+'searchlight_movie/ref.nii.gz')
subs = range(1, 18)
imgs = []
for sub in subs:
    sub_data = np.load(datadir+'/searchlight_movie/s%s.npy' % str(sub))
    img = new_img_like(ref_img, sub_data.astype(np.float))
    imgs.append(img)
imgs = concat_imgs(imgs)

## Get stats for real data

In [ ]:
x, y, z, t = imgs.shape
data = imgs.get_data()
statmap = ttest(np.moveaxis(data, -1, 0), 0).statistic
img = new_img_like(ref_img, statmap.astype(np.float))

## Do permutation correction

In [ ]:
real = img.get_data()
x, y, z = real.shape
pval = np.zeros_like(real)
zval = np.zeros_like(real)
d = img.get_data()
for i in range(x):
    for j in range(y):
        for k in range(z):
            if ~np.isnan(d[i,j,k]):
                pval[i, j, k] = np.sum(d[i,j,k] < sorted_perms[i,j,k,:])/100
                zval[i, j, k] = (d[i,j,k] - np.nanmean(sorted_perms[i,j,k,:]))/  np.nanstd(sorted_perms[i,j,k,:])

## Threshold

In [ ]:
zval[pval > .05]=0
zval[zval < 0]=0
zmap = new_img_like(ref_img, zval.astype(np.float))

## Plot video

In [ ]:
fsaverage = datasets.fetch_surf_fsaverage5()
tmap = zmap
texture_pr = surface.vol_to_surf(tmap, fsaverage.pial_right)
texture_pl = surface.vol_to_surf(tmap, fsaverage.pial_left)

In [ ]:
plotting.plot_surf_stat_map(fsaverage.infl_right, texture_pr, hemi='right',
                             bg_map=fsaverage.sulc_right, cmap='hot', threshold=.01, vmax=5)

In [ ]:
plotting.plot_surf_stat_map(fsaverage.infl_right, texture_pr, hemi='right',
                             bg_map=fsaverage.sulc_right,
                             view='medial', cmap='hot', threshold=.01, vmax=5)

In [ ]:
plotting.plot_surf_stat_map(fsaverage.infl_left, texture_pl, hemi='left',
                             bg_map=fsaverage.sulc_left, cmap='hot', threshold=.01, vmax=5)

In [ ]:
plotting.plot_surf_stat_map(fsaverage.infl_left, texture_pl, hemi='left',
                            bg_map=fsaverage.sulc_left, view='medial', cmap='hot', threshold=.01, vmax=5)

## Recall analysis

In [ ]:
perms = [load_img(datadir+'searchlight_recall/perm%s.nii' % str(perm)) for perm in range(1, 101)]
perms = concat_imgs(perms)
sorted_perms = np.sort(perms.get_data(), axis=3)

In [ ]:
ref_img = load_img(datadir+'searchlight_movie/ref.nii.gz')
subs = range(1, 18)
imgs = []
for sub in subs:
    sub_data = np.load(datadir + 'searchlight_recall/s%s.npy' % str(sub))
    img = new_img_like(ref_img, sub_data.astype(np.float))
    imgs.append(img)
imgs = concat_imgs(imgs)

In [ ]:
x, y, z, t = imgs.shape
data = imgs.get_data()
statmap = ttest(np.moveaxis(data, -1, 0), 0).statistic
img = new_img_like(ref_img, statmap.astype(np.float))

In [ ]:
real = img.get_data()
x, y, z = real.shape
pval = np.zeros_like(real)*np.nan
zval = np.zeros_like(real)*np.nan
d = img.get_data()
for i in range(x):
    for j in range(y):
        for k in range(z):
            if ~np.isnan(d[i,j,k]):
                pval[i, j, k] = np.sum(d[i,j,k] < sorted_perms[i,j,k,:])/100
                zval[i, j, k] = (d[i,j,k] - np.nanmean(sorted_perms[i,j,k,:]))/  np.nanstd(sorted_perms[i,j,k,:])

In [ ]:
zval[pval > .05]=0
zval[zval < 0]=0
zmap = new_img_like(ref_img, zval.astype(np.float))

In [ ]:
fsaverage = datasets.fetch_surf_fsaverage5()
tmap = zmap
texture_pr = surface.vol_to_surf(tmap, fsaverage.pial_right)
texture_pl = surface.vol_to_surf(tmap, fsaverage.pial_left)

In [ ]:
plotting.plot_surf_stat_map(fsaverage.infl_right, texture_pr, hemi='right',
                             bg_map=fsaverage.sulc_right, cmap='hot', threshold=.01, vmax=5)

In [ ]:
plotting.plot_surf_stat_map(fsaverage.infl_right, texture_pr, hemi='right',
                             bg_map=fsaverage.sulc_right,
                             view='medial', cmap='hot', threshold=.01, vmax=5)

In [ ]:
plotting.plot_surf_stat_map(fsaverage.infl_left, texture_pl, hemi='left',
                             bg_map=fsaverage.sulc_left, cmap='hot', threshold=.01, vmax=5)

In [ ]:
plotting.plot_surf_stat_map(fsaverage.infl_left, texture_pl, hemi='left',
                            bg_map=fsaverage.sulc_left, view='medial', cmap='hot', threshold=.01, vmax=5)